# ✅ Transfer Learning on CIFAR-10 (Kaggle Downloaded Version)
- Uses pre-downloaded CIFAR-10 (Python version)
- Loads it from a local directory
- Applies transfer learning with ResNet50
- Splits 80:20, trains, evaluates, visualizes

In [1]:
# 📦 Imports
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
# ⚙️ Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


In [3]:
# 📂 Path to your Kaggle-downloaded CIFAR-10 Python version
# Replace this path with where you've extracted it
data_root = r"C:\\Users\\lavan\\Downloads\\cifar-10-python-py"  # Update if needed

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset = torchvision.datasets.CIFAR10(
    root=data_root,
    train=True,
    download=False,
    transform=transform
)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)
class_names = dataset.classes
print("Classes:", class_names)

RuntimeError: Dataset not found or corrupted. You can use download=True to download it

In [ ]:
# 🔄 Load Pretrained ResNet50
resnet = models.resnet50(pretrained=True)
for param in resnet.parameters():
    param.requires_grad = False

resnet.fc = nn.Sequential(
    nn.Linear(resnet.fc.in_features, 256),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(256, 10)
)

resnet = resnet.to(device)

In [ ]:
# 🏋️ Train
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.fc.parameters(), lr=0.001)

epochs = 3
for epoch in range(epochs):
    resnet.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = resnet(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

In [ ]:
# ✅ Evaluate
correct = 0
total = 0
resnet.eval()
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = resnet(images)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
print(f"Test Accuracy: {100 * correct / total:.2f}%")

In [ ]:
# 📊 Visualize Predictions
def imshow(img):
    img = img / 2 + 0.5
    npimg = img.numpy().transpose((1, 2, 0))
    plt.imshow(npimg)
    plt.axis('off')
    plt.show()

dataiter = iter(test_loader)
images, labels = next(dataiter)
outputs = resnet(images.to(device))
_, preds = torch.max(outputs, 1)

for i in range(3):
    imshow(images[i])
    print(f"Predicted: {class_names[preds[i]]}, Actual: {class_names[labels[i]]}")